In [20]:
saved = {}

def backtrack(current_valve, valves, current_flow, current_time, visited, opened):
    possible_choices = valves[current_valve]['tunnels']
    visited.add(current_valve)
    if(f"{str(','.join(visited))}+{str(','.join(opened))}" not in saved.keys()):
        current_time += 1
        if (current_time >= 10):
            saved[f"{str(','.join(visited))}+{str(','.join(opened))}"] = current_flow
            return current_flow
        max_choices = [current_flow]
        for choice in possible_choices:
            max_choices.append(backtrack(choice, valves, current_flow, current_time, visited.copy(), opened.copy()))
        if(valves[current_valve]['flow'] == 0):
            saved[f"{str(','.join(visited))}+{str(','.join(opened))}"] = max(max_choices)
            return saved[f"{str(','.join(visited))}+{str(','.join(opened))}"]
        current_flow += valves[current_valve]['flow']
        opened.add(current_valve)
        if(f"{str(','.join(visited))}+{str(','.join(opened))}" not in saved.keys()):
            for choice in possible_choices:
                max_choices.append(backtrack(choice, valves, current_flow, current_time + 1, visited.copy(), opened.copy()))
            saved[f"{str(','.join(visited))}+{str(','.join(opened))}"] = max(max_choices)
            return saved[f"{str(','.join(visited))}+{str(','.join(opened))}"]
        else:
            print(current_valve, saved, "2")
            print()
            return saved[f"{str(','.join(visited))}+{str(','.join(opened))}"]
    else:
        print(current_valve, saved, "1")
        print()
        return saved[f"{str(','.join(visited))}+{str(','.join(opened))}"]

In [21]:
import re
with open('input.txt', 'r') as f:
    input = f.read()
input = input.splitlines()
regex = re.compile(r'Valve (.*) has flow rate=(\d+); tunnel(?:s?) lead(?:s?) to valve(?:s?) (.*)')
valves = {}
for line in input:
    valve, flow, tunnels = regex.match(line).groups()
    valves[valve] = {'flow': int(flow), 'tunnels': tunnels.split(', ')}
answer = backtrack('AA', valves, 0, 0, set(), set())
print(answer)

AA {'DD,CC,AA+': 0, 'DD,CC,BB,AA+': 0, 'DD,CC,AA+CC': 2, 'DD,CC,BB,AA+CC': 2} 1

AA {'DD,CC,AA+': 0, 'DD,CC,BB,AA+': 0, 'DD,CC,AA+CC': 2, 'DD,CC,BB,AA+CC': 2, 'CC,EE,DD,FF,AA+': 0, 'EE,DD,CC,AA+': 0, 'CC,EE,DD,FF,AA+EE': 3, 'EE,DD,CC,AA+EE': 3, 'DD,CC,AA+DD': 20} 1

BB {'DD,CC,AA+': 0, 'DD,CC,BB,AA+': 0, 'DD,CC,AA+CC': 2, 'DD,CC,BB,AA+CC': 2, 'CC,EE,DD,FF,AA+': 0, 'EE,DD,CC,AA+': 0, 'CC,EE,DD,FF,AA+EE': 3, 'EE,DD,CC,AA+EE': 3, 'DD,CC,AA+DD': 20, 'EE,DD,CC,AA+DD': 20} 1

CC {'DD,CC,AA+': 0, 'DD,CC,BB,AA+': 0, 'DD,CC,AA+CC': 2, 'DD,CC,BB,AA+CC': 2, 'CC,EE,DD,FF,AA+': 0, 'EE,DD,CC,AA+': 0, 'CC,EE,DD,FF,AA+EE': 3, 'EE,DD,CC,AA+EE': 3, 'DD,CC,AA+DD': 20, 'EE,DD,CC,AA+DD': 20} 2

AA {'DD,CC,AA+': 0, 'DD,CC,BB,AA+': 0, 'DD,CC,AA+CC': 2, 'DD,CC,BB,AA+CC': 2, 'CC,EE,DD,FF,AA+': 0, 'EE,DD,CC,AA+': 0, 'CC,EE,DD,FF,AA+EE': 3, 'EE,DD,CC,AA+EE': 3, 'DD,CC,AA+DD': 20, 'EE,DD,CC,AA+DD': 20} 1

EE {'DD,CC,AA+': 0, 'DD,CC,BB,AA+': 0, 'DD,CC,AA+CC': 2, 'DD,CC,BB,AA+CC': 2, 'CC,EE,DD,FF,AA+': 0, 'EE,DD,CC